In [21]:
!pip install substrate-interface

     |████████████████████████████████| 484 kB 473 kB/s eta 0:00:01
  Created wheel for tornado: filename=tornado-4.5.3-cp37-cp37m-macosx_10_15_x86_64.whl size=422378 sha256=1a5675ac70af319eca0ed1d55f0de242187b0f9a327d0d2cabbcf83b49f93e5b
  Stored in directory: /Users/const/Library/Caches/pip/wheels/a2/45/43/36ec7a893e16c1212a6b1505ded0a2d73cf8e863a0227c8e04
Successfully built tornado
  Attempting uninstall: tornado
    Found existing installation: tornado 6.0.4
    Uninstalling tornado-6.0.4:
      Successfully uninstalled tornado-6.0.4
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

notebook 6.1.4 requires tornado>=5.0, but you'll have tornado 4.5.3 which is incompatible.


In [46]:
from substrateinterface import SubstrateInterface

substrate = SubstrateInterface(
        url="http://127.0.0.1:9933",
        address_type=42,
        type_registry_preset='substrate-node-template'
    )

def current_block():
    result = substrate.get_runtime_block()
    current_block = result['block']['header']['parentHash']
    return current_block
print ('Current block:', current_block())


Current block: 0x96201d88bb4f5f63690292aa4743142f6f1c20d347ab785980362e9506862853


In [6]:
from substrateinterface import Keypair
alice_keypair = Keypair.create_from_uri('//Alice')
print ('Alice pubkey:', alice_keypair.public_key)
print ('Alice address:', alice_keypair.ss58_address)

Alice pubkey: 0xd43593c715fdd31c61141abd04a99fd6822c8558854ccde39a5684e7a56da27d
Alice address: 5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY


In [7]:
def get_balance(keypair):
    balance_info = substrate.get_runtime_state(
        module='System',
        storage_function='Account',
        params=[keypair.ss58_address],
        block_hash=current_block()
    ).get('result')
    return balance_info['data']['free'] / 10**12
print ('Alice balance:', get_balance(alice_keypair))

12404
Alice balance: 1151118.5004818426


In [8]:
from substrateinterface import SubstrateRequestException
import time
def transfer(keypair_A, keypair_B, ammount):
    call = substrate.compose_call(
        call_module='Balances',
        call_function='transfer',
        call_params={
            'dest': keypair_B.ss58_address,
            'value': ammount
        }
    )
    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair_A)
    return substrate.submit_extrinsic(extrinsic, wait_for_inclusion=False)
    
bob_keypair = Keypair.create_from_uri('//Bob')
alice_balance_1 = get_balance(alice_keypair)
bob_balance_1 = get_balance(bob_keypair)
print ('Alice balance:', alice_balance_1)
print ('Bob balance:', bob_balance_1, '\n')
amount = 100 * 10**12 + 1
print ('Transfer: Alice -> Bob: ', amount / 10**12)
extrinsic = transfer(alice_keypair, bob_keypair, amount)
print ('Wait for inclusion...')
time.sleep(15)
print ('Done. \n')
alice_balance_2 = get_balance(alice_keypair)
bob_balance_2 = get_balance(bob_keypair)
print ('Alice balance:', alice_balance_2)
print ('Bob balance:', bob_balance_2, '\n')
print ('Alice dif:', alice_balance_2 - alice_balance_1)
print ('Bob dif:', bob_balance_2 - bob_balance_1)


12405
12405
Alice balance: 1151118.5004818426
Bob balance: 1154724.504606847 

Transfer: Alice -> Bob:  100.000000000001
Wait for inclusion...
Done. 

12407
12407
Alice balance: 1151018.5003568423
Bob balance: 1154824.504606847 

Alice dif: -100.00012500025332
Bob dif: 100.0


In [9]:
substrate.get_metadata_storage_function('SubtensorModule', 'Something')

{'storage_name': 'Something',
 'storage_modifier': 'Optional',
 'storage_fallback_scale': '0x00',
 'storage_fallback': None,
 'documentation': '',
 'module_id': 'subtensormodule',
 'module_prefix': 'SubtensorModule',
 'module_name': 'SubtensorModule',
 'spec_version': 1,
 'type_key1': None,
 'type_key2': None,
 'type_hasher_key1': None,
 'type_hasher_key2': None,
 'type_value': 'u32',
 'type_is_linked': None,
 'type_class': 'PlainType'}

In [10]:
def setSomething(keypair, value):
    call = substrate.compose_call(
        call_module='SubtensorModule',
        call_function='do_something',
        call_params={'something': value}
    )
    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair)
    substrate.submit_extrinsic(extrinsic, wait_for_inclusion=False)
    print ('Wait for inclusion...')
    time.sleep(15)
    print ('Done. \n')
    
setSomething(alice_keypair, 12)

Wait for inclusion...


KeyboardInterrupt: 

In [163]:
def getSomething():
    result = substrate.get_runtime_state(
        module='SubtensorModule',
        storage_function='Something'
    ).get('result')
    return result
print (getSomething())

12


In [26]:
val = substrate.get_block_metadata(decode=True)
for mod in val.metadata.modules:
    if mod.name == "SubtensorModule":
        print (mod)
#         for stor in mod.storage.items:
#             if stor.name == "Weights":
#                 print (stor)

{'name': 'SubtensorModule', 'prefix': 'SubtensorModule', 'storage': {'prefix': 'SubtensorModule', 'items': [{'name': 'Weights', 'modifier': 'Default', 'type': {'DoubleMapType': {'hasher': 'Blake2_128Concat', 'key1': 'AccountId', 'key2': 'AccountId', 'value': 'u32', 'key2Hasher': 'Blake2_128Concat'}}, 'fallback': '0x00000000', 'docs': []}]}, 'calls': [{'name': 'set_weights', 'args': [{'name': 'dests', 'type': 'Vec<AccountId>'}, {'name': 'values', 'type': 'Vec<u32>'}], 'docs': [' An example dispatchable that takes a singles value as a parameter, writes the value to', ' storage and emits an event. This function must be dispatched by a signed extrinsic.']}], 'events': [{'name': 'WeightsSet', 'args': ['AccountId'], 'docs': [' Event documentation should end with an array that provides descriptive names for event', ' parameters. [something, who]']}], 'constants': [], 'errors': [{'name': 'NoneValue', 'docs': [' Error names should be descriptive.']}, {'name': 'StorageOverflow', 'docs': [' Error

In [15]:
substrate.get_metadata_storage_function('SubtensorModule', 'Weights')


{'storage_name': 'Weights',
 'storage_modifier': 'Default',
 'storage_fallback_scale': '0x00000000',
 'storage_fallback': 0,
 'documentation': '',
 'module_id': 'subtensormodule',
 'module_prefix': 'SubtensorModule',
 'module_name': 'SubtensorModule',
 'spec_version': 1,
 'type_key1': 'AccountId',
 'type_key2': 'AccountId',
 'type_hasher_key1': 'Blake2_128Concat',
 'type_hasher_key2': None,
 'type_value': 'u32',
 'type_is_linked': None,
 'type_class': 'DoubleMapType'}

In [36]:
def setWeights(keypair_A, keypair_B, value):
    call = substrate.compose_call(
        call_module='SubtensorModule',
        call_function='set_weights',
        call_params={'dests': [keypair_B.public_key], 'values': [value]}
    )
    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair_A)
    substrate.submit_extrinsic(extrinsic, wait_for_inclusion=False)
    print ('Wait for inclusion...')
    time.sleep(15)
    print ('Done. \n')

alice_keypair = Keypair.create_from_uri('//Alice')
bob_keypair = Keypair.create_from_uri('//Bob')
value = 123123

setWeights(alice_keypair, bob_keypair, value)

weights = substrate.get_runtime_state(
    module='SubtensorModule',
    storage_function='Weights',
    params=[alice_keypair.ss58_address, bob_keypair.ss58_address]
)

print (weights['result'])

Wait for inclusion...
Done. 

123123


In [47]:
weights = substrate.get_storage(
    module='SubtensorModule',
    function='state_getKeys',
    block_hash=current_block()
)

print (weights)

None


In [39]:
weights = substrate.get_runtime_state(
    module='SubtensorModule',
    storage_function='Weights',
    #params=[alice_keypair.ss58_address, bob_keypair.ss58_address]
)

print (weights)

ValueError: Storage call of type "DoubleMapType" requires 2 parameters

In [38]:
weights = substrate.iterate_map(
    module='SubtensorModule',
    storage_function='Weights',
)

ValueError: Given storage is not a map

In [20]:

def setWeight(keypair_A, keypair_B, value):
    call = substrate.compose_call(
        call_module='SubtensorModule',
        call_function='set_weight',
        call_params={'dest': keypair_B.ss58_address, 'value': 2321}
    )
    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair_A)
    substrate.submit_extrinsic(extrinsic, wait_for_inclusion=False)
    print ('Wait for inclusion...')
    time.sleep(15)
    print ('Done. \n')

alice_keypair = Keypair.create_from_uri('//Alice')
bob_keypair = Keypair.create_from_uri('//Bob')
value = 123123

setWeight(alice_keypair, bob_keypair, value)

weights = substrate.get_runtime_state(
    module='SubtensorModule',
    storage_function='Weights',
    params=[alice_keypair.ss58_address, bob_keypair.ss58_address]
)

assert weights['result'] == value

ValueError: Value should start with "0x" and should be 32 bytes long